In [1]:
from bs4 import BeautifulSoup 
import time
import requests.exceptions 
from urllib.parse import urlsplit 
from urllib.parse import urlparse 
from collections import deque

In [3]:
urls = ["https://www.lawyersnjurists.com/articlecat/bank-and-financial-institutes/",
        "https://www.lawyersnjurists.com/articlecat/business/",
        "https://www.lawyersnjurists.com/articlecat/environment-and-geography/",
        "https://www.lawyersnjurists.com/articlecat/garments-and-textile/",
        "https://www.lawyersnjurists.com/articlecat/general-health-and-medical-science/",
        "https://www.lawyersnjurists.com/articlecat/hotel-and-tourism/",
        "https://www.lawyersnjurists.com/articlecat/human-resource-and-managment/",
        "https://www.lawyersnjurists.com/articlecat/it-and-software/",
        "https://www.lawyersnjurists.com/articlecat/law-and-ethics/",
        "https://www.lawyersnjurists.com/articlecat/real-estate/",
        "https://www.lawyersnjurists.com/articlecat/reports-on-ngo/",
        "https://www.lawyersnjurists.com/articlecat/science-and-technology/",
        "https://www.lawyersnjurists.com/articlecat/social-and-political-science/",
        "https://www.lawyersnjurists.com/articlecat/telecommunication/",
        "https://www.lawyersnjurists.com/articles-and-reports"]
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
# url = "https://www.lawyersnjurists.com/"

# a queue of urls to be crawled next
new_urls = deque(urls)

# a set of urls that we have already processed
processed_urls = set()

# a set of domains inside the target website
local_urls = set()

# a set of domains outside the target website
foreign_urls = set()

# a set of broken urls
broken_urls = set()

In [3]:
with open("text_files/processed_urls.txt","r") as inputf:
    for line in inputf:
        processed_urls.add(line)

In [4]:
with open("text_files/useful_links.txt","r") as inputf:
    for line in inputf:
        new_urls.append(line)

In [5]:
# solving anti-flood issues using async function that requests at max 10 pages per minute
# async def geturl(url):
#     try: 
#         response = requests.get(url,headers=headers)
#     except(requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.InvalidSchema):
#         # add broken urls to it's own set, then continue
#         broken_urls.add(url)
#         return "-1"
#     return response
c=-1
# process urls one by one until we exhaust the queue
with open('text_files/processed_links.txt','a') as file:
    while len(new_urls):
        c+=1
        # move url from the queue to processed url set
        url = new_urls.popleft()
        file.write(url)
        processed_urls.add(url)
        # print the current url
        print(url)
        if(c%10==0):time.sleep(31)
        # catching any broken web pages and add them to the broken_urls set for later use
        try: 
            response = requests.get(url,headers=headers,timeout=10)
        except(requests.exceptions.MissingSchema, requests.exceptins.ConnectionError, requests.exceptions.InvalidSchema, requests.exceptions.InvalidURL):
            # add broken urls to it's own set, then continue
            broken_urls.add(url)
            continue

    #     response = geturl(url)
    #     if(response=="-1"):continue

        # extract base urlto resolve relative links
        parts = urlsplit(url)
        base = "{0.netloc}".format(parts)
        strip_base = base.replace("www.","")
        base_url = "{0.scheme}://{0.netloc}".format(parts)
        path = url[:url.rfind('/')+1] if '/' in parts.path else url


        soup = BeautifulSoup(response.text, "lxml")
        for link in soup.find_all('a'):
            # extract link url from the anchor
            anchor = link.attrs["href"] if "href" in link.attrs else ''
            if anchor.startswith('/'):
                local_link = base_url + anchor
                local_urls.add(local_link)
            elif strip_base in anchor:
                local_urls.add(anchor)
            elif not anchor.startswith('http'):
                local_link = path + anchor
                local_urls.add(local_link)
            else: foreign_urls.add(anchor)


        for i in local_urls:
            if not i in new_urls and not i in processed_urls and "webmail" not in i or "2095" not in i or "2096" not in i:
                new_urls.append(i)

https://www.lawyersnjurists.com/articlecat/bank-and-financial-institutes/


KeyboardInterrupt: 

In [ ]:
article_urls=[]
for i in processed_urls:
    if "https://www.lawyersnjurists.com/article/" in i:
        article_urls.append(i)
for i in article_urls:
    print(i)

In [ ]:
with open('text_files/links_final.txt','w') as f:
    for item in processed_urls:
        f.write("%s\n" % item)

In [ ]:
url = new_urls.popleft()
print(url)
print(type(url))